# Test the object detection task

reference: https://danielvanstrien.xyz/huggingface/huggingface-datasets/transformers/2022/08/16/detr-object-detection.html

In [12]:
from huggingface_hub import notebook_login

In [13]:
notebook_login()

## Dataset COCO

In [2]:
from datasets import load_dataset

In [14]:
dataset = load_dataset("detection-datasets/coco")

Found cached dataset parquet (/root/.cache/huggingface/datasets/detection-datasets___parquet/detection-datasets--coco-259ceb72df64b6a0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 117266
    })
    val: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 4952
    })
})

## Model DETR

In [17]:
model_checkpoint = "facebook/detr-resnet-50"
from transformers import DetrImageProcessor 

feature_extractor = DetrImageProcessor.from_pretrained(model_checkpoint)

In [18]:
from rich import inspect
inspect(feature_extractor, methods=True, dunder=True)

The `max_size` parameter is deprecated and will be removed in v4.27. Please specify in `size['longest_edge'] instead`.


╭─ DetrImageProcessor {   "do_normalize": true,   "do_pad": true,   "do_rescale": true,   "do_resize": true,   "f─╮
│ def (images, **kwargs) -> transformers.image_processing_utils.BatchFeature:                                     │
│                                                                                                                 │
│ Constructs a Detr image processor.                                                                              │
│                                                                                                                 │
│                        _auto_class = None                                                                       │
│                           __dict__ = {                                                                          │
│                                          '_processor_class': None,                                              │
│                                          'feature_extractor_type': 'DetrFeatureExtractor',                      │
│                                          'format': 'coco_detection',                                            │
│                                          'do_resize': True,                                                     │
│                                          'size': {'shortest_edge': 800, 'longest_edge': 1333},                  │
│                                          'resample': 2,                                                         │
│                                          'do_rescale': True,                                                    │
│                                          'rescale_factor': 0.00392156862745098,                                 │
│                                          'do_normalize': True,                                                  │
│                                          'image_mean': [0.485, 0.456, 0.406],                                   │
│                                          'image_std': [0.229, 0.224, 0.225],                                    │
│                                          'do_pad': True                                                         │
│                                      }                                                                          │
│                       do_normalize = True                                                                       │
│                             do_pad = True                                                                       │
│                         do_rescale = True                                                                       │
│                          do_resize = True                                                                       │
│                            __doc__ = '\n    Constructs a Detr image processor.\n\n    Args:\n        format     │
│                                      (`str`, *optional*, defaults to `"coco_detection"`):\n            Data     │
│                                      format of the annotations. One of "coco_detection" or "coco_panoptic".\n   │
│                                      do_resize (`bool`, *optional*, defaults to `True`):\n            Controls  │
│                                      whether to resize the image\'s `(height, width)` dimensions to the         │
│                                      specified `size`. Can be\n            overridden by the `do_resize`        │
│                                      parameter in the `preprocess` method.\n        size (`Dict[str, int]`      │
│                                      *optional*, defaults to `{"shortest_edge": 800, "longest_edge": 1333}`):\n │
│                                      Size of the image\'s `(height, width)` dimensions after resizing. Can be   │
│                                      overridden by the `size` parameter\n            in the `preprocess`        │
│                                      method.\n        

In [19]:
inspect(
    feature_extractor.__call__,
)

╭─ <bound method BaseImageProcessor.__call__ of DetrImageProcessor {   "do_normalize": true,   "do_pad": true,   ─╮
│ def BaseImageProcessor.__call__(images, **kwargs) -> transformers.image_processing_utils.BatchFeature:          │
│                                                                                                                 │
│ Preprocess an image or a batch of images.                                                                       │
│                                                                                                                 │
│ 27 attribute(s) not shown. Run inspect(inspect) for options.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [35]:
extracted_feature = feature_extractor(dataset['train'][0]['image'])

In [36]:
def transform_extract_feature(example_batch):
    return feature_extractor(example_batch['image'])

In [37]:
extracted_feature = dataset["train"].with_transform(transform_extract_feature)

In [42]:
from transformers import DetrForObjectDetection

model = DetrForObjectDetection.from_pretrained(
    model_checkpoint,
    ignore_mismatched_sizes=True,
)

In [47]:
from transformers import pipeline
pip = pipeline("object-detection", )

No model was supplied, defaulted to facebook/detr-resnet-50 and revision 2729413 (https://huggingface.co/facebook/detr-resnet-50).
Using a pipeline without specifying a model name and revision in production is not recommended.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [48]:
pip(dataset['train'][0]['image'])

[{'score': 0.9640734195709229,
  'label': 'orange',
  'box': {'xmin': 371, 'ymin': 40, 'xmax': 452, 'ymax': 90}},
 {'score': 0.9735672473907471,
  'label': 'orange',
  'box': {'xmin': 462, 'ymin': 41, 'xmax': 523, 'ymax': 89}},
 {'score': 0.9654533863067627,
  'label': 'bowl',
  'box': {'xmin': 0, 'ymin': 13, 'xmax': 429, 'ymax': 389}},
 {'score': 0.9886916875839233,
  'label': 'bowl',
  'box': {'xmin': 0, 'ymin': 190, 'xmax': 611, 'ymax': 473}},
 {'score': 0.9861542582511902,
  'label': 'orange',
  'box': {'xmin': 366, 'ymin': 0, 'xmax': 456, 'ymax': 67}},
 {'score': 0.9976679682731628,
  'label': 'broccoli',
  'box': {'xmin': 259, 'ymin': 231, 'xmax': 567, 'ymax': 473}},
 {'score': 0.9638203978538513,
  'label': 'orange',
  'box': {'xmin': 387, 'ymin': 70, 'xmax': 469, 'ymax': 143}},
 {'score': 0.9985100626945496,
  'label': 'bowl',
  'box': {'xmin': 313, 'ymin': 0, 'xmax': 631, 'ymax': 246}}]

In [49]:
inspect(pipeline)

╭───────────────────────────────────── <function pipeline at 0x7f39024dec20> ─────────────────────────────────────╮
│ def pipeline(task: str = None, model: Union[str, ForwardRef('PreTrainedModel'),                                 │
│ ForwardRef('TFPreTrainedModel'), NoneType] = None, config: Union[str,                                           │
│ transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str,                      │
│ transformers.tokenization_utils.PreTrainedTokenizer, ForwardRef('PreTrainedTokenizerFast'), NoneType] = None,   │
│ feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, image_processor:        │
│ Union[str, transformers.image_processing_utils.BaseImageProcessor, NoneType] = None, framework: Union[str,      │
│ NoneType] = None, revision: Union[str, NoneType] = None, use_fast: bool = True, use_auth_token: Union[bool,     │
│ str, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None,   │
│ torch_dtype=None, trust_remote_code: Union[bool, NoneType] = None, model_kwargs: Dict[str, Any] = None,         │
│ pipeline_class: Union[Any, NoneType] = None, **kwargs) -> transformers.pipelines.base.Pipeline:                 │
│                                                                                                                 │
│ Utility factory method to build a [`Pipeline`].                                                                 │
│                                                                                                                 │
│ 35 attribute(s) not shown. Run inspect(inspect) for options.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯